## Importing libraries

In [2]:
import os
import pandas as pd
import numpy as np

## Importing data 

In [5]:
current_directory = os.getcwd()
file_name = 'dataset_project_eHealth20232024.csv'
df = pd.read_csv(os.path.join(current_directory, file_name))

random_state = 10
print("datashape: ", df.shape)
print(df.info())
n_samples = df.shape[0]

datashape:  (160, 54)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160 entries, 0 to 159
Data columns (total 54 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   age        159 non-null    float64
 1   gender     160 non-null    int64  
 2   education  159 non-null    float64
 3   marital    160 non-null    int64  
 4   income     160 non-null    int64  
 5   phq_1      160 non-null    int64  
 6   phq_2      155 non-null    float64
 7   phq_3      159 non-null    float64
 8   phq_4      160 non-null    int64  
 9   phq_5      160 non-null    int64  
 10  phq_6      160 non-null    int64  
 11  phq_7      159 non-null    float64
 12  phq_8      159 non-null    float64
 13  phq_9      159 non-null    float64
 14  gad_1      159 non-null    float64
 15  gad_2      160 non-null    int64  
 16  gad_3      159 non-null    float64
 17  gad_4      159 non-null    float64
 18  gad_5      159 non-null    float64
 19  gad_6      159 non-null    f

# Data Cleaning

## Missing Data

In [6]:
# Sex and marital status -> average
# df_ccs['age'].fillna(df_ccs.mean()['age'])
# df_ccs['education'].fillna(df_ccs.median()['education'])
df['age'] = df['age'].fillna(df['age'].mean())
df['education'] = df['education'].fillna(df['education'].mean())

## Feature selection

In [7]:
# Feature Selection
columns_to_drop = df.columns[3:29] # Columns 5 to 41
columns_to_drop = columns_to_drop.insert(0, df.columns[1])
# columns_to_drop = columns_to_drop.append(df.columns[1])
# columns_to_drop = df.columns[0:42] # Columns 0 to 41
print(columns_to_drop)
df_ccs = df.drop(columns=columns_to_drop)
# df_ccs = df_ccs.dropna()
# print(df.head())
with pd.option_context('display.max_rows', 10,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(df_ccs)


Index(['gender', 'marital', 'income', 'phq_1', 'phq_2', 'phq_3', 'phq_4',
       'phq_5', 'phq_6', 'phq_7', 'phq_8', 'phq_9', 'gad_1', 'gad_2', 'gad_3',
       'gad_4', 'gad_5', 'gad_6', 'gad_7', 'eheals_1', 'eheals_2', 'eheals_3',
       'eheals_4', 'eheals_5', 'eheals_6', 'eheals_7', 'eheals_8'],
      dtype='object')
      age  education  heas_1  heas_2  heas_3  heas_4  heas_5  heas_6  heas_7  \
0    42.0        5.0     1.0     0.0     0.0       0       0       0       2   
1    51.0       18.0     0.0     1.0     2.0       0       0       2       0   
2    40.0        8.0     0.0     2.0     0.0       0       0       0       0   
3    19.0       25.0     0.0     0.0     3.0       3       3       3       3   
4    42.0       13.0     1.0     0.0     1.0       2       0       0       3   
..    ...        ...     ...     ...     ...     ...     ...     ...     ...   
155  29.0       18.0     3.0     3.0     3.0       2       2       3       3   
156  40.0       13.0     1.0     0.0  

## Reverse Scores

In [13]:
def reverse_score(string, max):
    # print(df_ccs[string])
    mean_value = (max)/2
    df_ccs[string] = df_ccs[string] - mean_value
    df_ccs[string] = - df_ccs[string]
    df_ccs[string] = df_ccs[string] + mean_value
    # print(df_ccs[string])

In [14]:
to_be_reversed = ['ccs_3', 'ccs_6', 'ccs_7', 'ccs_12', 'heas_1', 'heas_2', 'heas_3', 'heas_4', 'heas_5', 'heas_6', 'heas_7',
       'heas_8', 'heas_9', 'heas_10', 'heas_11', 'heas_12', 'heas_13']

for feature in to_be_reversed:
    if feature[:3] == 'ccs':
        reverse_score(feature, 6)
    else:
        reverse_score(feature, 3)

## Features Creation

In [16]:
data_ccs = pd.DataFrame() 
data_ccs['Age'] = df_ccs['age']
data_ccs['Education'] = df_ccs['education']
# data_ccs['Education'] = df_ccs['education']
data_ccs['Affective Symptoms'] = df_ccs[['heas_1', 'heas_2', 'heas_3', 'heas_4']].mean(axis=1)
data_ccs['Rumination'] = df_ccs[['heas_5', 'heas_6', 'heas_7']].mean(axis=1)
data_ccs['Behavioural Symptoms'] = df_ccs[['heas_8', 'heas_9', 'heas_10']].mean(axis=1)
data_ccs['Anxiety Personal Impact'] = df_ccs[['heas_11', 'heas_12', 'heas_13']].mean(axis=1)
data_ccs['Attribution Skepticism'] = df_ccs[['ccs_2', 'ccs_6', 'ccs_9']].mean(axis=1)
data_ccs['Impact Skepticism'] = df_ccs[['ccs_3', 'ccs_5', 'ccs_12']].mean(axis=1)
data_ccs['Trend Skepticism'] = df_ccs[['ccs_1', 'ccs_7', 'ccs_11']].mean(axis=1)
data_ccs['Response Skepticism'] = df_ccs[['ccs_4', 'ccs_8', 'ccs_10']].mean(axis=1)
# data_ccs['Income'] = df['income']
'''
dummies = pd.get_dummies(df.gender)
dummies = dummies.replace('True', 1)
dummies = dummies.replace('False', 0)
data_ccs['Male'] = dummies[0]
data_ccs['Female'] = dummies[1]
data_ccs['Non-binary'] = dummies[2]
dummies = pd.get_dummies(df.marital)
dummies = dummies.replace('True', 1)
dummies = dummies.replace('False', 0)
data_ccs['Single'] = dummies[0]
data_ccs['Married'] = dummies[1]
data_ccs['Divorced'] = dummies[2]
data_ccs['Widowed'] = dummies[3]
data_ccs['Separated'] = dummies[4]
'''
print(data_ccs)
data_ccs.to_csv('data_ccs.csv')


final_df = data_ccs.copy()
dummies = pd.get_dummies(df.gender)
dummies = dummies.replace('True', 1)
dummies = dummies.replace('False', 0)
final_df['Male'] = dummies[0]
final_df['Female'] = dummies[1]
final_df['Non-binary'] = dummies[2]
dummies = pd.get_dummies(df.marital)
dummies = dummies.replace('True', 1)
dummies = dummies.replace('False', 0)
final_df['Single'] = dummies[0]
final_df['Married'] = dummies[1]
final_df['Divorced'] = dummies[2]
final_df['Widowed'] = dummies[3]
final_df['Separated'] = dummies[4]
final_df['Income'] = df['income']
#print(final_df)
final_df.to_csv('final_df.csv')


      Age  Education  Affective Symptoms  Rumination  Behavioural Symptoms  \
0    42.0        5.0                2.75    2.333333              1.666667   
1    51.0       18.0                2.25    2.333333              3.000000   
2    40.0        8.0                2.50    3.000000              3.000000   
3    19.0       25.0                1.50    0.000000              2.333333   
4    42.0       13.0                2.00    2.000000              1.333333   
..    ...        ...                 ...         ...                   ...   
155  29.0       18.0                0.25    0.333333              0.000000   
156  40.0       13.0                1.75    2.000000              1.000000   
157  40.0       13.0                2.25    2.000000              2.666667   
158  41.0        8.0                2.75    2.666667              3.000000   
159  55.0        8.0                2.50    3.000000              3.000000   

     Anxiety Personal Impact  Attribution Skepticism  Impact Sk

# Exploratory Data Analysis